In [9]:
import boto3
import pandas as pd
import psycopg2
import json

In [10]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [11]:
KEY      = config.get('AWS','KEY')
SECRET   = config.get('AWS','SECRET')


DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES    = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE    = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER  =  config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                  =  config.get("DWH","DWH_DB")
DWH_DB_USER             =  config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD         =  config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT                =  config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME       =  config.get("DWH","DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)


('awsuser', 'Passw0rd123', 'myfirstdb')

In [12]:
pd.DataFrame({"Param":
                 ["DWH_CLUSTER_TYPE","DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER","DWH_DB", "DWH_DB_USER",
                  "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
               "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER,
                  DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
                  
                 })

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,my-first-redshift
4,DWH_DB,myfirstdb
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,Passw0rd123
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,s3-access


In [13]:
ec2 = boto3.resource('ec2',
                    region_name="eu-central-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

In [14]:
s3 = boto3.resource('s3',
                    region_name="eu-central-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )
iam = boto3.client('iam',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET,
                    region_name="eu-central-1"
                    )
redshift = boto3.client('redshift',
                        region_name="eu-central-1",
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                        )


In [15]:
bucket=s3.Bucket("aijaz-test")
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'corr.png',
 'date2008_pipe.txt',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [16]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [17]:
roleArn

'arn:aws:iam::453389129907:role/s3-access'

In [18]:
try:
    response = redshift.create_cluster(
    ClusterType = DWH_CLUSTER_TYPE,
    NodeType = DWH_NODE_TYPE,
        
    #Identifiers & Credentials
    DBName=DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    MasterUsername = DWH_DB_USER,
    MasterUserPassword=DWH_DB_PASSWORD,
        
    #Roles (for s3 access)
    IamRoles=[roleArn]
    )
    
    
except Exception as e:
    print(e)
    

In [21]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

{'ClusterIdentifier': 'my-first-redshift',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'ClusterAvailabilityStatus': 'Available',
 'MasterUsername': 'awsuser',
 'DBName': 'myfirstdb',
 'Endpoint': {'Address': 'my-first-redshift.cdqn5i8epsrl.eu-central-1.redshift.amazonaws.com',
  'Port': 5439},
 'ClusterCreateTime': datetime.datetime(2023, 2, 10, 21, 59, 23, 773000, tzinfo=tzutc()),
 'AutomatedSnapshotRetentionPeriod': 1,
 'ManualSnapshotRetentionPeriod': -1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-05d86cc7f82cfc836',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-0239fc511bd80c62d',
 'AvailabilityZone': 'eu-central-1c',
 'PreferredMaintenanceWindow': 'sun:07:00-sun:07:30',
 'PendingModifiedValues': {},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 1,
 'PubliclyA

In [22]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "VpcId"]
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

C:\Users\Maulwurf\AppData\Local\Temp\ipykernel_11992\1498625937.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value
0,ClusterIdentifier,my-first-redshift
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,myfirstdb
5,Endpoint,"{'Address': 'my-first-redshift.cdqn5i8epsrl.eu-central-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-0239fc511bd80c62d


In [23]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

In [24]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-0d1fd897b37d0061c')
